<a href="https://colab.research.google.com/github/jsilveira1409/CIVIL-459-Animal-Pose-Estimation/blob/sda/openpifpaf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
!pip3 install openpifpaf

import openpifpaf
import IPython
from openpifpaf.transforms import Crop
from PIL import Image
import gdown
import subprocess
openpifpaf.show.Canvas.show = True




[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: C:\Users\osour\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
!git clone --branch sda https://github.com/jsilveira1409/CIVIL-459-Animal-Pose-Estimation.git plugins
!mv plugins/openpifpaf_animalplugin/ openpifpaf_animalplugin


In [9]:
from openpifpaf_animalplugin import AnimalPoseEstimation
import coco_adapt as coco_adapt

!rm -rf data-animalpose/
!mkdir data-animalpose
!gdown "https://drive.google.com/drive/folders/1xxm6ZjfsDSmv6C9JvbgiGrmHktrUjV5x" -O data-animalpose --folder 
!unzip data-animalpose/images.zip -d data-animalpose/
!rm data-animalpose/images.zip

coco_adapt.adapt_to_coco()

rm: cannot remove 'data-animalpose/images.zipd4pgmzactmp': Device or resource busy
A subdirectory or file data-animalpose already exists.


Processing file 1Ge9jZppE9pGxqVyNAh6bWh4_ukffWBlF images.zip

Retrieving folder list
Retrieving folder list completed
Building directory structure
Downloading...
From (uriginal): https://drive.google.com/uc?id=1Ge9jZppE9pGxqVyNAh6bWh4_ukffWBlF
From (redirected): https://drive.google.com/uc?id=1Ge9jZppE9pGxqVyNAh6bWh4_ukffWBlF&confirm=t&uuid=ca926583-c0a8-4820-bd50-10bcffc9734a
To: c:\Users\osour\OneDrive - epfl.ch\EPFL\MA2\CIVIL-459 Deep Learning For Autonomous Vehicles\CIVIL-459-Animal-Pose-Estimation\data-animalpose\images.zip

  0%|          | 0.00/368M [00:00<?, ?B/s]
  0%|          | 524k/368M [00:00<04:35, 1.33MB/s]
  0%|          | 1.05M/368M [00:00<03:32, 1.73MB/s]
  0%|          | 1.57M/368M [00:00<03:12, 1.90MB/s]
  1%|          | 2.10M/368M [00:01<02:47, 2.18MB/s]
  1%|          | 2.62M/368M [00:01<02:29, 2.45MB/s]
  1%|          | 3.15M/368M [00:01<02:14, 2.70MB/s]
  1%|          | 3.67M/368M [00:01<02:03, 2.95MB/s]
  1%|          | 4.19M/368M [00:01<01:54, 3.16MB/s]
  1%|▏         | 4.72M/368M [00:01<01:46, 3.40MB/s]
  1%|▏         |


Processing file 1bAidtvR3MttetncXz0mfr47vYy7Uf5hm keypoints.json
Building directory structure completed



  error:  invalid compressed data to inflate
 bad CRC 00000000  (should be f371fc12)
file #2384:  bad zipfile offset (lseek):  218161152
file #2385:  bad zipfile offset (lseek):  218292224
file #2386:  bad zipfile offset (lseek):  218341376
file #2387:  bad zipfile offset (lseek):  218390528
file #2388:  bad zipfile offset (lseek):  218537984
file #2389:  bad zipfile offset (lseek):  218669056
file #2390:  bad zipfile offset (lseek):  218718208
file #2391:  bad zipfile offset (lseek):  218791936
file #2392:  bad zipfile offset (lseek):  218882048
file #2393:  bad zipfile offset (lseek):  218923008
file #2394:  bad zipfile offset (lseek):  219062272
file #2395:  bad zipfile offset (lseek):  219176960
file #2396:  bad zipfile offset (lseek):  219332608
file #2397:  bad zipfile offset (lseek):  219406336
file #2398:  bad zipfile offset (lseek):  219463680
file #2399:  bad zipfile offset (lseek):  219594752
file #2400:  bad zipfile offset (lseek):  219717632
file #2401:  bad zipfile offse

Archive:  data-animalpose/images.zip
   creating: data-animalpose/images/
  inflating: data-animalpose/images/2007_000063.jpg  
  inflating: data-animalpose/images/2007_000175.jpg  
  inflating: data-animalpose/images/2007_000332.jpg  
  inflating: data-animalpose/images/2007_000392.jpg  
  inflating: data-animalpose/images/2007_000464.jpg  
  inflating: data-animalpose/images/2007_000491.jpg  
  inflating: data-animalpose/images/2007_000504.jpg  
  inflating: data-animalpose/images/2007_000528.jpg  
  inflating: data-animalpose/images/2007_000549.jpg  
  inflating: data-animalpose/images/2007_000676.jpg  
  inflating: data-animalpose/images/2007_000720.jpg  
  inflating: data-animalpose/images/2007_000783.jpg  
  inflating: data-animalpose/images/2007_000799.jpg  
  inflating: data-animalpose/images/2007_000836.jpg  
  inflating: data-animalpose/images/2007_000876.jpg  
  inflating: data-animalpose/images/2007_000904.jpg  
  inflating: data-animalpose/images/2007_000925.jpg  
  inflat

Retrieving folder list
Retrieving folder list completed
Building directory structure
Downloading...
From (uriginal): https://drive.google.com/uc?id=1Ge9jZppE9pGxqVyNAh6bWh4_ukffWBlF
From (redirected): https://drive.google.com/uc?id=1Ge9jZppE9pGxqVyNAh6bWh4_ukffWBlF&confirm=t&uuid=7ac1b6e9-6f60-4735-8711-9d86ab0406e3
To: c:\Users\osour\OneDrive - epfl.ch\EPFL\MA2\CIVIL-459 Deep Learning For Autonomous Vehicles\CIVIL-459-Animal-Pose-Estimation\data-animalpose\images.zip

  0%|          | 0.00/368M [00:00<?, ?B/s]
  0%|          | 1.05M/368M [00:00<00:37, 9.71MB/s]
  1%|          | 2.10M/368M [00:00<00:41, 8.91MB/s]
  1%|          | 3.15M/368M [00:00<00:56, 6.45MB/s]
  1%|          | 4.19M/368M [00:00<00:54, 6.64MB/s]
  1%|▏         | 5.24M/368M [00:00<00:57, 6.25MB/s]
  2%|▏         | 6.29M/368M [00:00<00:59, 6.04MB/s]
  2%|▏         | 7.34M/368M [00:01<01:01, 5.90MB/s]
  2%|▏         | 8.39M/368M [00:01<01:01, 5.82MB/s]
  3%|▎         | 9.44M/368M [00:01<01:02, 5.78MB/s]
  3%|▎         

In [10]:

config = openpifpaf.plugin.register()
openpifpaf.DATAMODULES
dataset = AnimalPoseEstimation(image_dir='data-animalpose/images', annotation_file='data-animalpose/keypoints.json')

JSONDecodeError: Extra data: line 1 column 2358052 (char 2358051)

In [ ]:
!python3 -m openpifpaf.train \
    --dataset custom_animal \
    --basenet=shufflenetv2k16 \
    --lr=0.00002 \
    --momentum=0.95 \
    --epochs=10 \
    --lr-decay 160 260 \
    --lr-decay-epochs=10  \
    --weight-decay=1e-5 \
    --weight-decay=1e-5 \
    --val-interval 10 \
    --loader-workers 2 \
    --batch-size 1